# [class multiprocessing.Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Process)

```
进程对象代表单独运行在进程中的活动,Process class 与threading.Thread类有似的方法

构造函数调用时传入参数都是kwargs,group 总是为None,它的存在只是为了和threading.Thread.
target是可调用对象通过run()调用,默认为None,name是进程的名字,args是调用target函数时传入的元组参数,kwargs是传入的关键字参数,
deamon 设置守护进程的表示为 True or False,如果是默认的None,标识则继承自父类

如果一个子类,修改了构造函数,在对process进行任何操作之前确保调用了基类的__init__()
```
* run()
  ```
  Method representing the process’s activity.
  You may override this method in a subclass. The standard run() method invokes the callable object passed to the object’s constructor   as the target argument, if any, with sequential and keyword arguments taken from the args and kwargs arguments, respectively.
  ```
* start()
  ```
  开始进程的活动
  这个方法必须调用且最多调用一次,它保证对象的run()方法调用运行在单独的进程中
  ```
* join([timeout])
  ```
  如果可选参数timeout是None,join()的调用会阻塞至调用join()方法的进程的结束,如果timeout()为正数,进程最多阻塞timeout秒,如果线程结束或超时,join()返回None,检查process的exitcode以决定是否结束
  一个线程可以join()很多次
  一个线程不能join()自己,因为会产生死锁,不可在进程start之前join
  ```
* is_alive()
  ```
  通常,进程对象在start调用只是到子进程结束之前都是活着的
  ```
* deamon
  ```
  必须在start调用之前设置
  当一个进程结束时,会尝试终止其所有子进程
  daemonic process 不允许创建子进程,
  ```
* pid
  ```
  返回进程id
  ```
* exitcode
  ```
  如果进程还没有结束,属性值为none
  ```
* authkey
* sentinel
* terminate()
  ```
  终止进程,unix使用SIGTERM,windows使用TerminateProcess()
  子进程不会被终止,它们会变成孤儿线程
  ```
* kill()
  ```
  same as terminate,but using SIGKILL signal on Unix
  ```
* close()
  ```
  关闭process对象,释放所有相关资源,触发ValueError如果相关进程还是运行中,一旦close()成功返回,该对象多数方法,属性的访问都会触发ValueError
  ```

In [ ]:
import multiprocessing as mp


def p_name(a):
    print(mp.current_process().name, 'args:', a)
    time.sleep(2)


if __name__ == '__main__':
    print(mp.current_process().name)
    ps = []
    for i in range(10):
        p = mp.Process(target=p_name, name=f'test_process_{i}', args=(i,))
        p.start()
        ps.append(p)
    for i in ps:
        i.join()
    print('end')
'''
MainProcess
test_process_7 args: 7
test_process_0 args: 0
test_process_4 args: 4
test_process_3 args: 3
test_process_8 args: 8
test_process_5 args: 5
test_process_1 args: 1
test_process_9 args: 9
test_process_6 args: 6
test_process_2 args: 2
end
'''

In [ ]:
import multiprocessing as mp

class S_Process(mp.Process):
    def __init__(self,i):
        super().__init__()
        self.i = i

    def run(self):
        # print(mp.current_process() == self) # True
        print(self.name,self.i)
        time.sleep(2)


if __name__ == '__main__':
    print(mp.current_process().name)
    ps = []
    for i in range(10):
        p = S_Process(i)
        p.start()
        ps.append(p)
    for p in ps:
        p.join()
    print('end')
'''
S_Process-3 2
S_Process-7 6
S_Process-8 7
S_Process-4 3
S_Process-1 0
S_Process-6 5
S_Process-2 1
S_Process-5 4
S_Process-9 8
S_Process-10 9
end
'''

# [Connection Objects](https://docs.python.org/3/library/multiprocessing.html#connection-objects)

  > Connection 对象允许发送和接收可序列化对象或字符串,可以认为Connection 对象是面向消息的sockets连接
  > Connection 对象 通常通过使用pipe来创建

* # class multiprocessing.connection.Connection

  * send(obj)

    > 发送一个对象到connection的另一端,可以使用recv()读取

  * recv()

    > 返回另一端发送的对象,阻塞直到收到东西,如果没有剩下的东西等待接收,并且另一端已关闭,抛出EOFError

  * fileno()

    > return the file descriptor or handle used by the connection

  * close()

    > 关闭连接
    >
    > 当connection被垃圾回收时自动调用

  * poll([timeout])
    > 返回是否有数据可读取
    > 如果没传入timeout立即返回,如果timeout为正数,最多阻塞timeout秒,如果timeout是None则使用无限长的timeout

In [ ]:
import multiprocessing as mp
import time

a, b = mp.Pipe()
print(a)
print(b)
for i in range(10):
    a.send(i)
    b.send(i*i)
    time.sleep(0.1)
    print(b.recv(),b.poll(0.5))
print(b.fileno(),a.fileno())
b.close()

while True:
    try:
        print(a.recv())
    except EOFError:
        print('end')
        break
print(a)

# [Pipes and Queues](https://docs.python.org/3/library/multiprocessing.html#pipes-and-queues)
  ```
  在线程之间传递消息时,一个避免使用lock之类的同步原语
  可以使用Pipe()(连接两个进程) 或queue(允许多生产者消费者)
  Queue,SimpleQueue,JoinableQueue 都是多生产者多消费者,遵循FIFO(先进先出原则),以标准类queue.Queue 为蓝本

  如果使用JoinableQueue,在贝格任务从队列移除后必须调用JoinableQueue.task_done(),否则用于统计未完成任务的计数器会出现溢出
  ```
  # multiprocessing.Pipe([duplex])
  ```
  返回一对Connection对象(conn1,conn2),代表了pipe的两个端口.
  如果duplex = True (默认) 管道时双向的,duplex = False相反,conn1只能接受信息,conn2只能发送信息
  ```
  # class multiprocessing.Queue([maxsize])
  ```
  返回一个进程共享队列,通过pipe和一些locks/semaphores来实现,When a process first puts an item on the queue a feeder thread is started which transfers objects from a buffer into the pipe.
  ```
  * qsize()
    ```
    返回队列近似大小
    ```
  * empty()
    ```
    是否为空,和qsize一样不可信
    ```
  * full()
    ```
    是否满,不可信
    ```
  * put(obj[, block[, timeout]])
    ```
    把一个对象加入队列 ,如果 block=True(default) timeout=None(defalut),会阻塞直到空位可用,如果timeout为正数,最多阻塞timeout秒.超时抛出queue.Full exception,若block=False,若队列未满则入队,否则queue.Full exception
    ```
  * put_nowait(obj)
    ```
    same as put(obj,blcok=False)
    ```
  * get([block[, timeout]])
    ```
    从队列移除并返回一个任务,如果block=True(default), timeout=None( defalut),如有必要阻塞直至队列内有任务可获取,
    其余情况和put类似,exception 为queque.Empty
    ```
  * get_nowait()
    ```
    same as get(false)
    ```
  ```
  multiprocessing.Queue has a few additional methods not found in queue.Queue. These methods are usually unnecessary for most code:
  ```
  * close()
    ```
    Indicate that no more data will be put on this queue by the current process. The background thread will quit once it has flushed all buffered data to the pipe. This is called automatically when the queue is garbage collected.
    ```
  * join_thread()
  * cancel_join_thread()

In [ ]:
import multiprocessing as mp

def pipe_in(pipe,log):
    for i in range(3):
        pipe.send(f'data:{i},log:{log}')
        # print(f'data in {log} {i}')
    pipe.close()


def pipe_out(pipe):
        try:
            while True:
                msg = pipe.recv()
                print(f'msg {msg}')
        except EOFError:
            print('end')


if __name__ == '__main__':
    pipe_o,pipe_i = mp.Pipe(duplex=False)
    inp1 = mp.Process(target=pipe_in, args=(pipe_i,'a'))
    inp2 = mp.Process(target=pipe_in, args=(pipe_i,'b'))
    outp = mp.Process(target=pipe_out, args=(pipe_o,))
    inp1.start()
    inp2.start()
    pipe_i.close()
    outp.start()
'''
msg data:0,log:b
msg data:1,log:b
msg data:2,log:b
msg data:0,log:a
msg data:1,log:a
msg data:2,log:a
end
'''

In [ ]:
import multiprocessing as mp
import queue

class Producer(mp.Process):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue

    def run(self):
        for i in range(5):
            time.sleep(0.5)
            print('put',i)
            self.queue.put(i)


class Consumer(mp.Process):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue

    def run(self):
        while True:
            try:
                item = self.queue.get(timeout=2)
                print('get',item)
            except queue.Empty:
                print('队列空')
                break


if __name__ == '__main__':
    queue = mp.Queue(10)
    p1 = Consumer(queue)
    p2 = Producer(queue)
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print('end')
    
'''
put 0
get 0
put 1
get 1
put 2
get 2
put 3
get 3
put 4
get 4
队列空
end

'''